# Finder Camera 

This notebook is about exploring the ASICamera2 API for use with the ASI120MC-S as a finder camera for remote control and exploration of the night sky.

In [1]:
import asi

cameras = asi.getConnectedCameras()
print("Connected camera's:")

for camera_number in cameras:
    camera_name = (cameras[camera_number]['Name']).decode()
    print(f'  Camera Number: {camera_number} ({camera_name})')

Connected camera's:
  Camera Number: 0 (ZWO ASI120MC-S(Camera))
  Camera Number: 1 (ZWO ASI482MC)


### Playing with exposure and gain settings based on use (Finder Camera).
Natural daylight: exposure around 300,000 us. gain is 0

Night observing (decent image feedback): exposure around 2,000,000 us. gain around 80

Night observing (long exposure): exposure around 20,000,000 us. gain around 50

In [2]:
from finder_camera import FinderCamera
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import threading

app = FastAPI()

camera: FinderCamera = None

def run():
    uvicorn.run(app, host="0.0.0.0")
      
def start_api():
    _api_thread = threading.Thread(target=run)
    _api_thread.start()
    
def create_stream(): 
    global camera
    camera = FinderCamera({ 
        'Camera ID': 0, 
        'Exposure_Time_In_us': 200 * 1000, 
        'Gain': 82, 
        'WhiteBalance_Red': 55, 
        'WhiteBalance_Blue': 77,
        'Height': 960,
        'Width': 1280,
        'Mode': 'RGB24',
        'Overlay_Filename': 'overlays/alignment-cross-hair.png',
        'Overly_Height_Offset': -20,
        'Overly_Width_Offset': -18
    })     
    while True:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + bytearray(camera.get_frame()) + b'\r\n\r\n')
      
@app.get("/")
def get_video_stream():
    return StreamingResponse(create_stream(), media_type='multipart/x-mixed-replace; boundary=frame')

start_api()

INFO:     Started server process [19763]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [3]:
%%HTML 
<img src="http://192.168.0.30:8000/"/>

INFO:     192.168.0.11:45422 - "GET / HTTP/1.1" 200 OK
